## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [69]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.input_dim=28*28
        pnum1=128
        pnum2=10
        self.W1 = tf.Variable(tf.random.truncated_normal([self.input_dim, pnum1], stddev=0.02))
        self.b1 = tf.Variable(tf.ones([pnum1]))
        self.W2 = tf.Variable(tf.random.truncated_normal([pnum1, pnum2], stddev=0.03))
        self.b2 = tf.Variable(tf.ones([pnum2]))
        self.Relu = layers.ReLU()
        
        
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, self.input_dim])
        x = tf.matmul(x, self.W1) + self.b1
        x = self.Relu(x)
        x = tf.matmul(x, self.W2) + self.b2
        logits = self.Relu(x)
        
        
    
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [70]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [77]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 0.93591386 ; accuracy 0.80728334
epoch 1 : loss 0.9345013 ; accuracy 0.8075167
epoch 2 : loss 0.9330937 ; accuracy 0.80771667
epoch 3 : loss 0.9316913 ; accuracy 0.808
epoch 4 : loss 0.93029404 ; accuracy 0.80826664
epoch 5 : loss 0.9289017 ; accuracy 0.8084
epoch 6 : loss 0.9275143 ; accuracy 0.8086
epoch 7 : loss 0.926132 ; accuracy 0.80883336
epoch 8 : loss 0.9247547 ; accuracy 0.8089667
epoch 9 : loss 0.9233823 ; accuracy 0.80916667
epoch 10 : loss 0.92201483 ; accuracy 0.8093
epoch 11 : loss 0.9206524 ; accuracy 0.80946666
epoch 12 : loss 0.9192948 ; accuracy 0.8096167
epoch 13 : loss 0.9179421 ; accuracy 0.80973333
epoch 14 : loss 0.91659427 ; accuracy 0.80995
epoch 15 : loss 0.9152513 ; accuracy 0.81016666
epoch 16 : loss 0.91391313 ; accuracy 0.81035
epoch 17 : loss 0.91257983 ; accuracy 0.81045
epoch 18 : loss 0.9112513 ; accuracy 0.81056666
epoch 19 : loss 0.9099275 ; accuracy 0.8107167
epoch 20 : loss 0.90860844 ; accuracy 0.8108
epoch 21 : loss 0.9072942 ; ac